# Хеширование  

Множество (set) - добавление/удаление ключей, проверка принадлежности.  
Словарь (dictionary, map) или ассоциативная таблица (assotiated table) - каждому ключу сопоставлены данные, ключи уникальны.

Предположим, нам надо хранить набор активных ip адресов (множество) или время последнего захода ip адресов (словарь). Для простоты пусть множество. 172.16.254.1 - это четыре 8-битных числа, можно считать одно 32-битное число (2^32) -> 2886794753. Выделим 2^32 бит памяти и отметим бит 2886794753. Это называется **прямая адресация**. Доступ О(1), самый быстрый вариант, но памяти не напасешься.

### Хеш-функция

    K - множество ключей
    h: K -> {0..m-1} - х.ф.
    h(k) - значение х.ф. для ключа k, хеш ключа k
    

Коллизия k1 != k2: h(k1) == h(k2) н ключу указатель на (односвязный) спискок элементов, у которых есть коллизия (одинаковый ключ). Как записная книжка, букв m, под каждой буквой список записей. O(l) - поиск, O(1) - добавление, O(l) - удаление (l - длина списка). Задача, чтобы элементы были максимально равномерно распределены по ключам (длины списков близко друг к другу, тогда l = n / m).

### Второй способ. Открытая адресация  

Только при n <= m.  

    h: {h0: K -> {0..m-1}} -> {0..m-1} - отображает каждое из m уникальных значений функции h0 в еще m уникальных значений функции h.  
        т.е. h(k, 0) .. h(k, m-1), перестановка 0,1..m-1.
    например h(k,j) = (h0(k) + j) % m

Адреса крутятся покругу для каждого k,j. Хеш - это ссылка на ячейку памяти, содержащую ключ. При удалении нам надо не затирать ячейку, а писать флаг пропуска, чтоб не разрывать цепочку, и можно было искать дальше.

### Примеры стандартных хэш-функций

    1. K=[0,1) действительные числа, h(k) = round(m*k)
    2. K=Z целые числа, h(k) = k % m или h(k) = round( m * (c*k % 1) ), c=const
    3. K={ последовательность чисел }, h(a0..at) = (a0 + a1*x + a2*x^2 + ... + a t-1 x^(t-1)) mod m, 
            x - фиксированное основание

Все они дают m различных значений для ключа.





## Вероятностный анализ алгоритмов хеширования  

Хотим, чтобы хэш-функция была детерминирована, не давала разных значений на одинаковых ключах. Приходит ключ, мы вычисляем по нему значение хэш-функции, кладем в хэш-таблицу (или соответств. цепочку хэш-таблицы). Потом когда придет другой ключ и мы вычислим хэш, он должен указывать на то же место в таблице (памяти).  

    Теорема: предположим, что h(k) выбирается равномерно случайно из 0..m-1. Тогда средняя длина цепочки будет n/m, среднее время поиска O(1+n/m). 

Это и есть оптимальный случай, который уже нельзя улучшить. Логично, что для ускорения нужно увеличивать m и их равномерность для каждого конкретного набора ключей. Доказывается тем, что матожидание для каждого ключа 1/m, сумма матожиданий n/m. 
Если не повезло и ключ не угадался - просматривается цепочка уже оптимальной длины n/m.  
Оценка вероятности попасть 1/m. Выводится формула матожидания суммы удачных и неудачных поисков, вжухх, получаем 1+n/m.     

## Универсальное хеширование: определение  

Пусть есть семейство хэш-функций "мощности" m. Оно универсально, если любых двух различных ключей, доля таких функций из семейства, на которых h(k1) == h(k2) в общем количестве функций семейства не больше 1/m.  

    Теорема: среднее время поиска для универсального хеширования O(1+n/m)  
Доказывается тем, что матожидание для попадания в такой случай <= (n - 1) / m , а до полной вероятности добавляется 1 для случая когда угадали. Получаем 1+n/m

**Хотим минимальной время поиска, но и минимальный объем памяти для лишних ключей m. Тут разумно соблюдать баланс.  
Обычно выбирается пороговое значение n/m = 0.9, когда ключи подходят к этому порогу, они перевыделяются с увеличением пространства в 2 раза.**

## Универсальное хеширование: конструкция  

Они СУЩЕСТВУЮТ! Считаем ключами просто числа.  

    Теорема: пусть K = {0..t-1} и p >= t. Пусть hab(k) = ((ak + b) mod p) mod m.
    Тогда можество таких хэш-функций hab: 1<=a<=(p-1), 0<=b<=(p-1) является универсальным.  

Т.е. доля таких a b которые для двух разных k дадут одинаковое значение hab не более чем 1/m.
Доказывается примерно так  

    t1 = (ak1 + b) mod p
    t2 = (ak2 + b) mod p
    t1 != t2: если t1 == t2 -> a(k1 - k2) mod p == 0 -> но a < p -> значит (k1 - k2) mod p == 0 -> но они тоже меньше p  

А значит есть биекция (a, b) <-> (t1, t2): t1 != t2 и оба < p (взаимнооднозначное соответствие).  
Тогда P2(hab(t1) == hab(t2)) == P2(t1 == t2 mod m), т.е. такое будет только если t2 делится на m.
Количество разных t менее чем p(p-1), а разных t которые равны по модулю m будет p(p-1)/m. 
Т.е. вероятность выбрать такие a b чтоб t1 == t2 будет 1 / m.  
Так шо вот.


## Поиск образца в тексте  

Вход: тект T, образец P2  
Выход: все позиции i: 0<=i<=|T|-|P|, T\[i..i+|P|-1] == P  

        012345678   
    T = ababacaba, P = aba
    выход: 0, 2, 6 (c перекрытием)

Наивный подход - посимвольно двигать образец по тексту. O(|P|(|T|-|P|)) = O(|T||P|). Долго и не эффективно, особенно если T и P однообразные и отличаются только в конце.  

## Алгоритм Карпа–Рабина  

Если по уму...  
Сравниваем хеш-функцию образца с хеш-функцией от кусков текста. Какая же функция должна быть? Она должна зависеть от всех символов в окне просмотра, и должна работать хорошо с изменением окна (добавлением / удалением символа). 

Полиномиальная х.ф. 

    T[i+1..i+|P|] окно i+1 ..  i+|P|
    T[i..i+|P|+1] окно i ..  i+|P|+1  

    h(i+1) - полином степени |P|-1 c коэффициентами из T от i+1 до i+|P|. Все по модулю p (каждая операция)
    h(i) - полином степени |P|-1 c коэффициентами из T от i до i+|P|+1. Все по модулю p (каждая операция)

Оставляем только изменения начала и хвоста и получаем рекуррентное выражение  

    h(i) = ( (h(i+1) - t(i+|P|)*x^(|P|-1) )*x + t(i)*x^0 (mod p)

Пересчет за конст. время. 

Считаем с конца (иначе в рекурр. выражении нужно будет делить на x (причем по модулю), а не умножать).   

Самые долгие вычисления только на певом окне. Использовать кеширование степений иксов.  

! Степени высокие, считать только mod p  

## Псевдокод  

    Зафиксировать большое p  
    выбрать случайное x: 1<=x<=(p-1)
    вычислить x^(|P|-1) mod p и h(p)
    идя справа налево по T 
        вычислить и сохранить все хеши всех окон |P| 
    для каждого окна W в T 
        если h(W) = h(P)  
            сравнить W и P посимвольно 

Время работы при больших p: O(|T| + occ |P|), где occ число вхожение P в T где необходимо будет посимвольное сравнение 
Ложное срабатывание (когда W!=P при h(W)=h(P)), это когда выбранный x оказался корнем полиномиальной х.ф. степени p с константой от предыдущего хеша. Ложные срабатывания будут не чаще |P|/p.

Но выше p - выше сложность вычисления.

# Задача. Телефонная книга  

Реализовать структуру данных, эффективно обрабатывающую запросы вида add number name, del number и find number.  
Вход. Последовательность запросов вида add number name, del number и find number, где number — телефонный номер, содержащий не более семи знаков, а name — короткая строка.  
Выход. Для каждого запроса find number выведите соответствующее имя или сообщите, что такой записи нет.  


In [5]:
SAMPLES = "12\nadd 911 police\nadd 76213 Mom\nadd 17239 Bob\nfind 76213\nfind 910\nfind 911\ndel 910\ndel 911\nfind 911\nfind 76213\nadd 76213 daddy\nfind 76213",
OUTPUTS = ...
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

M_MAX = 10 ** 7

def _hash(key):
    return int(key) % M_MAX

def _find(*args):
    print(hash_table[_hash(args[0])] or 'not found')

def _add(*args):
    print(f"{args[0]=} {_hash(args[0])=} {args}" )
    hash_table[_hash(args[0])] = args[1]

def _del(*args):
    hash_table[_hash(args[0])] = None

def run():
    commands = {'add': _add, 'find': _find, 'del': _del}
    for _ in range(int(input())):
        cmd, *args = input().split()
        commands[cmd](*args)

hash_table = [None] * M_MAX
run()

args[0]='911' _hash(args[0])=911 ('911', 'police')
args[0]='76213' _hash(args[0])=76213 ('76213', 'Mom')
args[0]='17239' _hash(args[0])=17239 ('17239', 'Bob')
Mom
not found
police
not found
Mom
args[0]='76213' _hash(args[0])=76213 ('76213', 'daddy')
daddy


# Задача. Хеширование цепочками

Формат входа. Первая строка размер хеш-таблицы m. Следующая строка содержит количество запросов n. Каждая из последующих n строк содержит запрос одного из перечисленных выше четырёх типов.  
Формат выхода. Для каждого из запросов типа find и check выведите результат в отдельной строке.  
Ограничения. 1 ≤ n ≤ 10^5 ; n/5 ≤ m ≤ n. Все строки имеют длину от одного до пятнадцати и содержат только буквы латинского алфавита.  

In [1]:
SAMPLES = "5\n12\nadd world\nadd HellO\ncheck 4\nfind World\nfind world\ndel world\ncheck 4\ndel HellO\nadd luck\nadd GooD\ncheck 2\ndel good",
OUTPUTS = ...
READER = (x for x in SAMPLES[0].split('\n')); input = lambda: next(READER)

p = 10 ** 9 + 7     # prime
x = 263

def run():
    def _hash(key):
        h = 0
        for i, ch in enumerate(key):
            h += ord(ch) * x ** i
        return h % p % m

    def find(key):
        for i, val in enumerate(hash_table[_hash(key)]):
            if val == key:
                return i
        return -1

    def _find(key):
        print('yes' if find(key) != -1 else 'no')

    def _add(key):
        if find(key) == -1:
            hash_table[_hash(key)].insert(0, key)

    def _del(key):
        if find(key) != -1:
            hash_table[_hash(key)].pop(find(key))

    def _check(i):
        print(*hash_table[int(i)])

    commands = {'add': _add, 'find': _find, 'del': _del, 'check': _check}
    m = int(input())
    n = int(input())
    hash_table = [list() for _ in range(m)]
    for _ in range(n):
        cmd, arg = input().split()
        commands[cmd](arg)

run()

HellO world
no
yes
HellO
GooD luck


# Поиск образца в тексте. Поиск Карпа-Рабина

Найти все вхождения строки Pattern в строку Text.
Вход. Строки Pattern и Text.  
Выход. Все индексы i строки Text, начиная с которых строка Pattern входит в Text:

    Text[i..i + |Pattern| − 1] = Pattern.

Ограничения. 1 ≤ |Pattern| ≤ |Text| ≤ 5 · 10^5. Суммарная длина всех вхождений образца в текста не превосходит 10^8 . Обе строки содержат буквы латинского алфавита.


In [6]:
SAMPLES = "aba\nabacaba", "Test\ntestTesttesT", "aaaaa\nbaaaaaaa",
OUTPUTS = "0 4", "4", "1 2 3",
READER = (x for x in SAMPLES[2].split('\n')); input = lambda: next(READER)


P = 10 ** 9 + 7
x = 263

def _hash(key):
    h = 0
    curr_x = prev_x = 1
    for ch in key:
        h += ord(ch) * curr_x % P
        curr_x, prev_x = curr_x * x % P, curr_x
    return h % P, prev_x

def _delta_hash(ch_in, ch_out, old_hash, power_x):
    h = (old_hash - ord(ch_out)*power_x) * x + ord(ch_in)
    return h % P 

def steps(s, d):
    for i in range(len(s), d, -1):
        yield i - 1  - d, s[i - 1  - d], s[i - 1]

def check(pat, txt, i):
    for pi, ch in enumerate(pat):
        if ch != txt[i+pi]:
            print('collision!')
            return False
    return True

def searchKR(pattern, text):
    lenp, lent = len(pattern), len(text)
    ph, px = _hash(pattern)
    W = []

    h = ph
    for (i, ch_in, ch_out) in steps(text + pattern, lenp):
        h = _delta_hash(ch_in, ch_out, h, px)
        if i < lent - lenp + 1 and ph == h and check(pattern, text, i):
            W.append(i)
    return W


pattern, text = input(), input()
print(*reversed(searchKR(pattern, text)))


1 2 3


In [8]:
import random
import string

P = 10 ** 9 + 7
x = 101


n = 10 ** 6
text = "".join(random.choice(string.ascii_letters) for _ in range(n))
pattern = "aba"
print(*reversed(searchKR(pattern, text)))

8299 51416 391693 598023
